<a href="https://colab.research.google.com/github/WillNovus/Tasks/blob/main/RoBertA_Enron_AA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Feature extraction
import email
from sklearn.feature_extraction.text import TfidfVectorizer
from string import punctuation
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize,sent_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.util import ngrams
from collections import Counter
nltk.download('stopwords')
nltk.download('punkt')
import heapq


# Classification
import xgboost as xgb
import lightgbm as lgbm
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# Parallelize apply on Pandas
!pip install pandarallel
from pandarallel import pandarallel
pandarallel.initialize()

# Evaluation
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

#Optimization
!pip install optuna
import optuna
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

!git clone https://github.com/NVIDIA/apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex
!pip install transformers
!pip install --upgrade tqdm
!pip install tensorboardX
!pip install simpletransformers
from simpletransformers.classification import ClassificationModel

import torch

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 2.8 MB/s eta 0:00:00
  Created wheel for pandarallel: filename=pandarallel-1.6.5-py3-none-any.whl size=16672 sha256=5672fef2f1d26b50e76388089ac0431ad507ff38d3c2509ecd04ee99e67b6b90
  Stored in directory: /root/.cache/pip/wheels/50/4f/1e/34e057bb868842209f1623f195b74fd7eda229308a7352d47f
Successfully built pandarallel
INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.7 MB/s eta 0:00:00
Cloning into 'apex'...
remote: Enumerating objects: 11289, done.
remote: Counting objects: 100% (3426/3426), done.
remote: Compressing objects: 100% (362/362), done.
remote: Total 11289 (delta 31

In [2]:
# Set working directory
from google.colab import drive
drive.mount('/content/drive')
import os
root_dir = "/content/drive/My Drive/"
data = "Colab Notebooks/data/"
def create_and_set_working_directory(data):
  if os.path.isdir(root_dir + data) == False:
    os.mkdir(root_dir + data)
    print(root_dir + data + ' did not exist but was created.')
  os.chdir(root_dir + data)
  !touch 'new_file_in_working_directory.txt'
  print('\nYour working directory was changed to ' + root_dir + data + \
        "\n\nAn empty text file was created there. You can also run !pwd to confirm the current working directory.")

create_and_set_working_directory(data)

Mounted at /content/drive

Your working directory was changed to /content/drive/My Drive/Colab Notebooks/data/

An empty text file was created there. You can also run !pwd to confirm the current working directory.


In [5]:
#Load the preprocessed dataset
df = pd.read_csv('enron_stylo (1).csv')

In [6]:
list_authors = list(pd.DataFrame(df['From'].value_counts()[:11]).reset_index()['index'])

In [7]:
list_authors

['kay.mann@enron.com',
 'sara.shackleton@enron.com',
 'jeff.dasovich@enron.com',
 'chris.germany@enron.com',
 'vince.kaminski@enron.com',
 'tana.jones@enron.com',
 'sally.beck@enron.com',
 'kate.symes@enron.com',
 'drew.fossum@enron.com',
 'eric.bass@enron.com',
 'matthew.lenhart@enron.com']

In [8]:
sub_df = df[df['From'].isin(list_authors)]

In [9]:
dict_nlp_enron = {}
k=0
for val in np.unique(sub_df.From):
    dict_nlp_enron[val] = k
    k += 1
sub_df['Target'] = sub_df['From'].apply(lambda x: dict_nlp_enron[x])

<ipython-input-9-180dd1246881>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['Target'] = sub_df['From'].apply(lambda x: dict_nlp_enron[x])


In [10]:
substr_df = train_test_split(sub_df['content'], sub_df['Target'], test_size=0.2, stratify=sub_df['Target'])
substr_train = list(substr_df[0].index)
substr_test = list(substr_df[1].index)

dfx_train = sub_df.loc[substr_train]
dfx_test = sub_df.loc[substr_test]


In [11]:
#RoBertA + classification layer
roberta_model = ClassificationModel('roberta', 'roberta-base', num_labels=11, args={'reprocess_input_data': True, 'overwrite_output_dir': True,  'num_train_epochs' : 10}, use_cuda=True)
roberta_model.train_model(dfx_train[['content', 'Target']])

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/19880 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/2485 [00:00<?, ?it/s]

...

Running Epoch 6 of 10:   0%|          | 0/2485 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/2485 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/2485 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/2485 [00:00<?, ?it/s]

(24850, 0.14395251248204252)

In [13]:
roberta_predictions, roberta_raw_outputs = roberta_model.predict(list(dfx_test['content']))
print(roberta_predictions[700])
actuals = list(dfx_test['Target'])
print(actuals[700])

  0%|          | 0/4970 [00:00<?, ?it/s]

  0%|          | 0/622 [00:00<?, ?it/s]

0
0


In [14]:
roberta_score = accuracy_score(roberta_predictions, dfx_test['Target'])
f1_roberta = f1_score(roberta_predictions, dfx_test['Target'], average="macro")
print("The accuracy of Roberta on Enron is {}".format(roberta_score))
print("The F1-score of Roberta is {}".format(f1_roberta))

The accuracy of Roberta on Enron is 0.971830985915493
The F1-score of Roberta is 0.9709521912646747


In [15]:
#Style based classifier
dfx_stylo_train = dfx_train[["avg_len", "num_short_w", "per_digit", "per_cap", "f_a", "f_b", "f_c", "f_d", "f_e", "f_f", "f_g", "f_h", "f_i", "f_j", "f_k", "f_l", "f_m", "f_n", "f_o", "f_p", "f_q", "f_r", "f_s", "f_t", "f_u", "f_v", "f_w", "f_x", "f_y", "f_z", "f_0", "f_1", "f_2", "f_3", "f_4", "f_5", "f_6", "f_7", "f_8", "f_9", "f_e_0", "f_e_1", "f_e_2", "f_e_3", "f_e_4", "f_e_5", "f_e_6", "f_e_7", "f_e_8", "f_e_9", "f_e_10", "f_e_11", "richness", "sentiment_score"]]
dfx_stylo_test = dfx_test[["avg_len", "num_short_w", "per_digit", "per_cap", "f_a", "f_b", "f_c", "f_d", "f_e", "f_f", "f_g", "f_h", "f_i", "f_j", "f_k", "f_l", "f_m", "f_n", "f_o", "f_p", "f_q", "f_r", "f_s", "f_t", "f_u", "f_v", "f_w", "f_x", "f_y", "f_z", "f_0", "f_1", "f_2", "f_3", "f_4", "f_5", "f_6", "f_7", "f_8", "f_9", "f_e_0", "f_e_1", "f_e_2", "f_e_3", "f_e_4", "f_e_5", "f_e_6", "f_e_7", "f_e_8", "f_e_9", "f_e_10", "f_e_11", "richness", "sentiment_score"]]

In [16]:
#Xgboost
xg_model = xgb.XGBClassifier().fit(dfx_stylo_train, dfx_train['Target'])
y_pred_xg = xg_model.predict(dfx_stylo_test)
y_proba_xg = xg_model.predict_proba(dfx_stylo_test)
y_proba_train_xg = xg_model.predict_proba(dfx_stylo_train)

In [17]:
score_style = accuracy_score(dfx_test['Target'], y_pred_xg)
f1_style = f1_score(dfx_test['Target'], y_pred_xg, average="macro")
print("The accuracy of xgb classifier on enron author style is {}".format(score_style))
print("The F1-score of xgb classifier on enron author style is {}".format(f1_style))

The accuracy of xgb classifier on enron author style is 0.8378269617706238
The F1-score of xgb classifier on enron author style is 0.8440710041201512


In [27]:
#Model Combination/ (RoBertA + XgBoost) -> XGBoost
train_predictions, roberta_raw_out_train = roberta_model.predict(list(dfx_train['content']))
feat_ROBERTA_XG_train = np.concatenate([roberta_raw_out_train, y_proba_train_xg], axis=1)
feat_ROBERTA_XG_test = np.concatenate([roberta_raw_outputs, y_proba_xg], axis=1)
clf_comb = xgb.XGBClassifier().fit(feat_ROBERTA_XG_train, dfx_train['Target'])
y_pred_comb = clf_comb.predict(feat_ROBERTA_XG_test)
score_comb = accuracy_score(dfx_test['Target'], y_pred_comb)
f1_comb = f1_score(dfx_test['Target'], y_pred_comb, average="macro")

print("Training done, accuracy is : ", score_comb)
print("Training done, f1-score is : ", f1_comb)

  0%|          | 0/19880 [00:00<?, ?it/s]

  0%|          | 0/2485 [00:00<?, ?it/s]

Training done, accuracy is :  0.9716297786720322
Training done, f1-score is :  0.9703432155709447


In [23]:
  #Functions to extract n_gram features from text data. #Adapted from Sari et al., Topic or Style? Exploring the Most Useful Features for Authorship Attribution
  def return_best_bi_grams(text):
      bigrams = ngrams(text,2)

      data = dict(Counter(bigrams))
      list_ngrams = heapq.nlargest(100, data.keys(), key=lambda k: data[k])
      return list_ngrams

  def return_best_tri_grams(text):
      trigrams = ngrams(text,3)

      data = dict(Counter(trigrams))
      list_ngrams = heapq.nlargest(100, data.keys(), key=lambda k: data[k])
      return list_ngrams

  def find_freq_n_gram_in_txt(text, list_bigram, list_trigram):

      to_ret = []

      num_bigrams = len(Counter(zip(text,text[1:])))
      num_trigrams = len(Counter(zip(text,text[1:],text[2:])))

      for n_gram in list_bigram:
          to_ret.append(text.count(''.join(n_gram))/num_bigrams)

      for n_gram in list_trigram:
          to_ret.append(text.count(''.join(n_gram))/num_trigrams)

      return to_ret


In [24]:
text = " ".join(sub_df['content'].values)
list_bigram = return_best_bi_grams(text)
list_trigram = return_best_tri_grams(text)

feats_train = dfx_train['content'].apply(lambda x: find_freq_n_gram_in_txt(x, list_bigram, list_trigram)).values
feats_test = dfx_test['content'].apply(lambda x: find_freq_n_gram_in_txt(x, list_bigram, list_trigram)).values

feats_train = pd.DataFrame(feats_train)[0].apply(lambda x: pd.Series(x))
feats_test = pd.DataFrame(feats_test)[0].apply(lambda x: pd.Series(x))

In [25]:
xg_ngrams = xgb.XGBClassifier().fit(feats_train, dfx_train['Target'])


In [26]:
#Xgboost
y_pred_grams = xg_ngrams.predict(feats_test)
y_proba_grams = xg_ngrams.predict_proba(feats_test)
y_proba_train_grams = xg_ngrams.predict_proba(feats_train)
score_char = accuracy_score(dfx_test['Target'], y_pred_grams)
f1_char = f1_score(dfx_test['Target'], y_pred_grams, average="macro")
print("Training done, accuracy is : ", score_char)
print("Training done, f1-score is : ", f1_char)

Training done, accuracy is :  0.8724346076458752
Training done, f1-score is :  0.8683901561235416


In [29]:
# RoBERta + Style + Char

feat_for_BERT_full_train = np.concatenate([feat_ROBERTA_XG_train, y_proba_train_grams], axis=1)
feat_for_BERT_full_test = np.concatenate([feat_ROBERTA_XG_test, y_proba_grams], axis=1)

full_comb = LogisticRegression(random_state=0).fit(feat_for_BERT_full_train, dfx_train['Target'])
y_pred = full_comb.predict(feat_for_BERT_full_test)
score_comb_fin = accuracy_score(dfx_test['Target'], y_pred)
f1_comb_fin = f1_score(dfx_test['Target'], y_pred, average="macro")
print("Training done, accuracy is : ", score_comb_fin)
print("Training done, f1-score is : ", f1_comb_fin)

Training done, accuracy is :  0.971830985915493
Training done, f1-score is :  0.9709521912646747
